# Install libraries

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install keybert

In [3]:
from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import List, Tuple
import re
from collections import defaultdict
from keybert import KeyBERT
from tqdm.notebook import tqdm
import time
import pickle
from datetime import datetime
import os
import pandas as pd

# Load dataset SemEval

In [4]:
# Load dataset
ds = load_dataset("SemEvalWorkshop/sem_eval_2010_task_8")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.23k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/673k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/231k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2717 [00:00<?, ? examples/s]

## See some samples

In [5]:
# Take first 10 samples for testing
test_samples = list(ds['train'].select(range(10)))

# Display first sample structure
print("Sample data structure:")
print(test_samples[0])

# Display all 10 sentences and their entities
print("\nSamples with their entities:")
for i, sample in enumerate(test_samples):
    entities = re.findall(r'<e[12]>(.*?)</e[12]>', sample['sentence'])
    print(f"\n{i+1}. Sentence: {sample['sentence']}")
    print(f"   Entities: {entities}")

Sample data structure:
{'sentence': 'The system as described above has its greatest application in an arrayed <e1>configuration</e1> of antenna <e2>elements</e2>.', 'relation': 3}

Samples with their entities:

1. Sentence: The system as described above has its greatest application in an arrayed <e1>configuration</e1> of antenna <e2>elements</e2>.
   Entities: ['configuration', 'elements']

2. Sentence: The <e1>child</e1> was carefully wrapped and bound into the <e2>cradle</e2> by means of a cord.
   Entities: ['child', 'cradle']

3. Sentence: The <e1>author</e1> of a keygen uses a <e2>disassembler</e2> to look at the raw assembly code.
   Entities: ['author', 'disassembler']

4. Sentence: A misty <e1>ridge</e1> uprises from the <e2>surge</e2>.
   Entities: ['ridge', 'surge']

5. Sentence: The <e1>student</e1> <e2>association</e2> is the voice of the undergraduate student population of the State University of New York at Buffalo.
   Entities: ['student', 'association']

6. Sentence: Th

# Load model & define functions

In [6]:
def clean_sentence(text: str) -> str:
    """Remove XML tags from text"""
    return re.sub(r'</?e[12]>', '', text).strip()

def extract_keyphrases(text: str, model: KeyBERT, top_n: int = 2) -> List[str]:
    """Extract keyphrases using KeyBERT"""
    keyphrases = model.extract_keywords(text,
                                      keyphrase_ngram_range=(1, 2),
                                      stop_words='english',
                                      top_n=top_n,
                                      use_maxsum=True)
    return [k[0] for k in keyphrases]

# Initialize model
print("Loading KeyBERT model...")
model = KeyBERT()

Loading KeyBERT model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Check the samples

In [7]:
# Process test samples
for i, sample in enumerate(test_samples):
    # Get original sentence and true entities
    sentence = sample['sentence']
    true_entities = re.findall(r'<e[12]>(.*?)</e[12]>', sentence)

    # Clean sentence and extract keyphrases
    clean_text = clean_sentence(sentence)
    extracted_phrases = extract_keyphrases(clean_text, model)

    # Print results
    print(f"\nSample {i+1}:")
    print(f"Clean text: {clean_text}")
    print(f"True entities: {true_entities}")
    print(f"Extracted keyphrases: {extracted_phrases}")
    print("-" * 80)


Sample 1:
Clean text: The system as described above has its greatest application in an arrayed configuration of antenna elements.
True entities: ['configuration', 'elements']
Extracted keyphrases: ['greatest', 'antenna elements']
--------------------------------------------------------------------------------

Sample 2:
Clean text: The child was carefully wrapped and bound into the cradle by means of a cord.
True entities: ['child', 'cradle']
Extracted keyphrases: ['means', 'child carefully']
--------------------------------------------------------------------------------

Sample 3:
Clean text: The author of a keygen uses a disassembler to look at the raw assembly code.
True entities: ['author', 'disassembler']
Extracted keyphrases: ['look raw', 'assembly code']
--------------------------------------------------------------------------------

Sample 4:
Clean text: A misty ridge uprises from the surge.
True entities: ['ridge', 'surge']
Extracted keyphrases: ['misty', 'uprises surge']
-

## Evaluation method

In [8]:
from typing import List, Tuple

def evaluate_matches(true_entities: List[str], extracted_phrases: List[str], partial_match: bool = False) -> Tuple[float, float, float]:
    """Calculate precision, recall, and F1 score."""

    matched_true = set()
    matched_extracted = set()

    if partial_match:
        # Allow partial matches (case insensitive)
        for ext in extracted_phrases:
            for gold in true_entities:
                if (ext.lower() in gold.lower() or gold.lower() in ext.lower()):
                    matched_extracted.add(ext)
                    matched_true.add(gold)
    else:
        # Require exact matches (case insensitive)
        for ext in extracted_phrases:
            for gold in true_entities:
                if ext.lower() == gold.lower():
                    matched_extracted.add(ext)
                    matched_true.add(gold)

    matches = len(matched_true)  # Count unique matches

    precision = matches / len(extracted_phrases) if extracted_phrases else 0
    recall = matches / len(true_entities) if true_entities else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1



## Evaluate all samples

In [9]:
all_metrics = []
for i, sample in enumerate(test_samples):
    # Get entities and extract keyphrases
    true_entities = re.findall(r'<e[12]>(.*?)</e[12]>', sample['sentence'])
    clean_text = clean_sentence(sample['sentence'])
    extracted_phrases = extract_keyphrases(clean_text, model)

    # Calculate metrics
    precision, recall, f1 = evaluate_matches(true_entities, extracted_phrases)
    all_metrics.append((precision, recall, f1))

    # Print results
    print(f"\nSample {i+1}:")
    print(f"True entities: {true_entities}")
    print(f"Extracted phrases: {extracted_phrases}")
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1: {f1:.2f}")

# Calculate average metrics
avg_precision = np.mean([m[0] for m in all_metrics])
avg_recall = np.mean([m[1] for m in all_metrics])
avg_f1 = np.mean([m[2] for m in all_metrics])

print("\nAverage Metrics:")
print(f"Precision: {avg_precision:.2f}")
print(f"Recall: {avg_recall:.2f}")
print(f"F1 Score: {avg_f1:.2f}")


Sample 1:
True entities: ['configuration', 'elements']
Extracted phrases: ['greatest', 'antenna elements']
Precision: 0.00, Recall: 0.00, F1: 0.00

Sample 2:
True entities: ['child', 'cradle']
Extracted phrases: ['means', 'child carefully']
Precision: 0.00, Recall: 0.00, F1: 0.00

Sample 3:
True entities: ['author', 'disassembler']
Extracted phrases: ['look raw', 'assembly code']
Precision: 0.00, Recall: 0.00, F1: 0.00

Sample 4:
True entities: ['ridge', 'surge']
Extracted phrases: ['misty', 'uprises surge']
Precision: 0.00, Recall: 0.00, F1: 0.00

Sample 5:
True entities: ['student', 'association']
Extracted phrases: ['association voice', 'student population']
Precision: 0.00, Recall: 0.00, F1: 0.00

Sample 6:
True entities: ['complex', 'producer']
Extracted phrases: ['sprawling complex', 'producer silver']
Precision: 0.00, Recall: 0.00, F1: 0.00

Sample 7:
True entities: ['inflammation', 'infection']
Extracted phrases: ['acid production', 'inflammation distal']
Precision: 0.00, Reca

## Process dataset and save results

In [10]:
def process_dataset_in_batches(mode, ngram, nr_cand, dataset, model, batch_size=200, save_every=1000):
    """
    Process the dataset in batches
    """
    start_time = time.time()
    all_metrics = []
    all_results = []
    total_samples = len(dataset)

    # Create progress bar
    pbar = tqdm(total=total_samples, desc="Processing samples")

    for i in range(0, total_samples, batch_size):
        # Get batch
        batch = dataset.select(range(i, min(i + batch_size, total_samples)))

        # Process each sample in batch
        for sample in batch:
            true_entities = re.findall(r'<e[12]>(.*?)</e[12]>', sample['sentence'])
            clean_text = clean_sentence(sample['sentence'])
            extracted_phrases = extract_keyphrases(clean_text, model)

            # Calculate metrics
            precision, recall, f1 = evaluate_matches(true_entities, extracted_phrases)
            batch_metrics.append((precision, recall, f1))

            # Store detailed results
            batch_results.append({
                # 'sentence': sample['sentence'],
                # 'true_entities': true_entities,
                # 'extracted_phrases': extracted_phrases,
                'mode' : mode,
                'ngram' : ngram,
                'nr_cand' : nr_cand,
                'precision': precision,
                'recall': recall,
                'f1'  : f1
                # 'metrics': {'precision': precision, 'recall': recall, 'f1': f1}
            })

        # Update main lists
        all_metrics.extend(batch_metrics)
        all_results.extend(batch_results)

        # Update progress bar
        pbar.update(len(batch))

    pbar.close()

    # Calculate final averages
    avg_metrics = calculate_average_metrics(all_metrics)

    # Print time taken
    time_taken = time.time() - start_time
    print(f"\nTotal time taken: {time_taken:.2f} seconds")

    save(mode, ngram, nr_cand, avg_metrics, time_taken)

    return all_results, avg_metrics

def save(mode, ngram, nr_cand, avg_metrics, time):
    """Save results to CSV file, appending new rows if the file exists."""
    # Define the filename
    filename = "/content/drive/MyDrive/NLP_Proj/Results/Baseline_results.csv"

    # Prepare data for DataFrame
    data = {
               'mode' : mode,
                'ngram' : ngram,
                'nr_cand' : nr_cand,
                'avg_metrics' : avg_metrics,
                'time' : time
    }

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Check if the file exists to determine mode
    if os.path.exists(filename):
        # Append to existing CSV
        df.to_csv(filename, mode='a', header=False, index=False)
    else:
        # Create a new CSV file with header
        df.to_csv(filename, mode='w', header=True, index=False)

    print(f"\nSaved results to {filename}")

def calculate_average_metrics(metrics):
    """Calculate average metrics from list of (precision, recall, f1) tuples"""
    avg_precision = np.mean([m[0] for m in metrics])
    avg_recall = np.mean([m[1] for m in metrics])
    avg_f1 = np.mean([m[2] for m in metrics])

    return {
        'precision': avg_precision,
        'recall': avg_recall,
        'f1': avg_f1
    }

# Process the full training dataset

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
print("Starting full dataset processing...")
results, avg_metrics = process_dataset_in_batches('Train', '(1,2)', ' ', ds['train'], model)

print("\nFinal Average Metrics:")
print(f"Precision: {avg_metrics['precision']:.3f}")
print(f"Recall: {avg_metrics['recall']:.3f}")
print(f"F1 Score: {avg_metrics['f1']:.3f}")

Starting full dataset processing...


Processing samples:   0%|          | 0/8000 [00:00<?, ?it/s]


Total time taken: 196.61 seconds

Saved results to /content/drive/MyDrive/NLP_Proj/Results/Baseline_results.csv

Final Average Metrics:
Precision: 0.069
Recall: 0.069
F1 Score: 0.069


# Process the test dataset


In [13]:
print("Starting test dataset processing...")
test_results, test_metrics = process_dataset_in_batches('Test', '(1,2)', ' ', ds['test'], model)


print("\nTest Set Metrics:")
print(f"Precision: {test_metrics['precision']:.3f}")
print(f"Recall: {test_metrics['recall']:.3f}")
print(f"F1 Score: {test_metrics['f1']:.3f}")

Starting test dataset processing...


Processing samples:   0%|          | 0/2717 [00:00<?, ?it/s]


Total time taken: 63.85 seconds

Saved results to /content/drive/MyDrive/NLP_Proj/Results/Baseline_results.csv

Test Set Metrics:
Precision: 0.071
Recall: 0.071
F1 Score: 0.071


# Print results

In [14]:
from tabulate import tabulate

# Collecting results from our experiments
def create_comparison_table(sem_eval_train, sem_eval_test):
    data = [
        ['SemEval-2010 Task 8', 'Train',
         sem_eval_train['precision'],
         sem_eval_train['recall'],
         sem_eval_train['f1']],
        ['SemEval-2010 Task 8', 'Test',
         sem_eval_test['precision'],
         sem_eval_test['recall'],
         sem_eval_test['f1']]
    ]

    headers = ['Dataset', 'Split', 'Precision', 'Recall', 'F1 Score']
    print("\nBaseline Metrics:")
    print(tabulate(data, headers=headers, tablefmt='pipe', floatfmt='.3f'))

# Use our actual metrics from previous runs
create_comparison_table(
    sem_eval_train=avg_metrics,      # from SemEval train processing
    sem_eval_test=test_metrics      # from SemEval test processing

)


Baseline Metrics:
| Dataset             | Split   |   Precision |   Recall |   F1 Score |
|:--------------------|:--------|------------:|---------:|-----------:|
| SemEval-2010 Task 8 | Train   |       0.069 |    0.069 |      0.069 |
| SemEval-2010 Task 8 | Test    |       0.071 |    0.071 |      0.071 |
